In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Načtení a zpracování denních dat
denni_excel = (
    pd.ExcelFile("data-daily.xlsx")
    .parse("Praha-Ruzyne")
    .set_index("time")
    .drop(columns=["wdir"])
    .dropna(subset=["tavg"])
)

**Otázka:** Co vrátí `denni_ruzyne_index.isna().count()`?

In [ ]:
denni_ruzyne_index.isna().count()

**Úkol:** Zkuste odstranit všechny řádky, pro které není definován `tsun`. Co nám to říká?

In [ ]:
denni_ruzyne.dropna(subset=["tsun"])

**Úkol:** Zkuste odstranit všechny řádky, pro které není definován `snow`. Co nám to říká?

In [ ]:
denni_ruzyne.dropna(subset=["snow"])

**Úkol:** Nejstarší tag Pandas na https://github.com/pandas-dev/pandas je verze 0.3.0 z 20. února 2011. Jaké bylo v ten den v Praze - Ruzyni počasí?

In [ ]:
denni_ruzyne.loc["2011-02-20"]

**Úkol:** Jaká byla průměrná teplota první (a jedinou) neděli v roce 2010, která byla zároveň začátkem měsíce? Pokud máte řešení a čas, zkuste vymyslet aternativní způsob(y). 

In [ ]:
# odkomentuj a doplň
denni_ruzyne.loc[
    (denni_ruzyne.index.year == 2010)
    & (denni_ruzyne.index.day == 1)
    & (denni_ruzyne.index.day_name() == "Sunday"),
    "tavg",
]

In [ ]:
sns.catplot(
    data=denni_ruzyne[["tavg", "tmin", "tmax"]],
    orient="h",
    kind="box",
    aspect=2,
);

**Úkol**: Doplňte pomocné sloupce `season` a `significant_precipitation` (jistě uhádnete jakého pandas-typu budou :). První definuje roční období (jen jednoduše podle kalendářních měsíců), druhý označuje dny, kdy byly srážky vyšší než v **90 %** všech dní v našich datech (můžete zkusit i jiný limit).

1. Porovnejte numericky základní statistiky celého datasetu a podmnožiny, kdy výrazně pršelo nebo sněžilo. Zvyšují v průměru srážky teplotu? A co maximální nebo minimální? A jak je to se standardní odchylkou?
2. Použijte `sns.catplot` pro vizuální srovnání distribučních funkcí pro jednotlivá roční období a dny s málo / hodně srážkami.

In [ ]:
# odkomentuj a doplň
season = denni_ruzyne.index.month.map({
    1: "zima",
    2: "zima",
    3: "jaro",
    4: "jaro",
    5: "jaro",
    6: "léto",
    7: "léto",
    8: "léto",
    9: "podzim",
    10: "podzim",
    11: "podzim",
    12: "zima"
})

significant_precipitation = denni_ruzyne["prcp"] > denni_ruzyne["prcp"].quantile(.9)
significant_precipitation

In [ ]:
# úkol - jednoduché srovnání statistik pomocí rozdílu
(
    denni_ruzyne.loc[significant_precipitation].describe() - 
    denni_ruzyne.describe()
)

In [ ]:
# úkol - vizuální srovnání statistik
sns.catplot(
     data=denni_ruzyne.assign(
        significant_precipitation=significant_precipitation,
         season=season,
     ),
    kind="box",
     aspect=2,
    hue="significant_precipitation",
     y="season",
    x="tavg",
);

Co že jsme to vlastně vytvořili?

In [ ]:
rocni_ruzyne = denni_ruzyne.resample("1YE")

**Úkol:** Navrhněte vhodnou roční agregaci pro maximální teplotu (ne průměr) a vykreslete.

In [ ]:
# odkomentuj a doplň
rocni_ruzyne["tmax"].max().plot();

**Úkol:** Najděte, kdy v období 2000-2025 napršelo nejvíc za týden (tedy za průběžných 7 dnů). Odpovídá to nějaké vaší historické zkušenosti?

In [ ]:
# doplňte nebo vyřešte po svém :)
denni_ruzyne.loc["2000":"2025"]["prcp"].rolling("7D").sum().sort_values(ascending=False).iloc[:10]

**Nepovinný úkol:** Rozšiřte hledání na celé období v naší datové sadě - jsou některé hodnoty realistické?

In [ ]:
denni_ruzyne["prcp"].rolling("7D").sum().sort_values(ascending=False).iloc[:10]

In [ ]:
hodinove_ruzyne = (
    pd.read_parquet("praha-meteostat.parquet")
    .assign(
        time=hodinove_ruzyne_original["time"].dt.tz_localize("UTC").dt.tz_convert("Europe/Prague")
    )
    .set_index("time")
)

**Úkol:** Jak silný foukal vítr (jak moc pršelo) v kterou hodinu den tvého narození?

In [ ]:
hodinove_ruzyne.loc["2000-01-01", "wspd"]